In [1]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 5.50 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [4]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [5]:
def split_df(df):
    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [6]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-07 00:09:17.763024: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 00:09:19.852437: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 00:09:19.852529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 00:09:20.156374: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 00:09:20.827438: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 00:09:20.828562: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

In [8]:
import pickle
from itertools import product

class Tester:
  def __init__(self, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input_25, self.output_25 = split_df(df_25)
    self.input_1000, self.output_1000 = split_df(df_1000)
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=2,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)
  
  def LoadDataset(self, fileName, trainer=None):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'rb') as f:
      _train, _vt, _val, _test = pickle.load(f)
    trainer._train, trainer._vt, trainer._val, trainer._test = _train, _vt, _val, _test

  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= boundarie): # should be >= to acsending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.SaveFigs(trainer, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input_25, self.output_25, self.input_1000, self.output_1000)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults('mse_sup', header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)



# Treinando com 25 dados


In [9]:
tester = Tester(run_times=25, dataset_run_times=10)

In [10]:
tester.Loop(sort_by='mse_sup',
            boundarie=2.0,
            hidden_sizes = [[10, 11], [11, 12], [13, 11]],
            regularizers=[0.02, 0.2, 0.5],
            learning_rate=[0.01, 0.1])

Testando combinacao1: Hidden Size=[10, 11], regularizer=0.02, learning_rate=0.01
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  3.1158997719999206
Stopped at epoch:  85
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.2694019539999317
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.16063374799978192
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.1626135860001341
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.21720688400000654
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.17241878000004363
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 7 +++++

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_1_6,0.865892,0.709554,0.493692,0.868528,0.694366,0.562823,1.942215,0.656830,0.154888,0.405859
model_1_1_5,0.864661,0.708362,0.508143,0.871854,0.702900,0.567988,1.950184,0.638083,0.150970,0.394527
model_1_1_4,0.863382,0.707133,0.522528,0.875119,0.711375,0.573356,1.958403,0.619421,0.147123,0.383272
model_1_1_3,0.862053,0.705865,0.536838,0.878320,0.719786,0.578933,1.966879,0.600857,0.143352,0.372104
model_1_1_2,0.860673,0.704559,0.551061,0.881455,0.728123,0.584727,1.975618,0.582406,0.139659,0.361032
model_1_1_1,0.859239,0.703211,0.565186,0.884519,0.736382,0.590744,1.984627,0.564082,0.136049,0.350066
model_1_1_0,0.857750,0.701823,0.579199,0.887509,0.744553,0.596991,1.993914,0.545903,0.132527,0.339215


DataFrame salvo em ../results/metrics_1_1
+++++++++++ [1_2] | 1 ++++++++++++++++++
Elapsed time:  0.782398338999883
Stopped at epoch:  31
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_2] | 2 ++++++++++++++++++
Elapsed time:  0.21494112700020196
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_2] | 3 ++++++++++++++++++
Elapsed time:  0.2137496549999014
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_2] | 4 ++++++++++++++++++
Elapsed time:  0.16070002700007535
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_2] | 5 ++++++++++++++++++
Elapsed time:  0.16721802700021726
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_2] | 6 ++++++++++++++++++
Elapsed time:  0.15621183699977337
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_2] | 7 ++++++++++++++++++
Elapsed time:  0.16241859

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_2_24,0.889945,0.729086,0.558371,0.860166,0.783628,0.461878,1.811602,1.445008,0.807904,1.126456
model_1_2_23,0.889891,0.729040,0.561486,0.860382,0.784727,0.462105,1.811914,1.434814,0.806657,1.120735
model_1_2_22,0.889828,0.728984,0.564629,0.860587,0.785828,0.462369,1.812283,1.424532,0.805469,1.115000
model_1_2_21,0.889755,0.728920,0.567797,0.860782,0.786932,0.462673,1.812712,1.414165,0.804345,1.109255
model_1_2_20,0.889673,0.728847,0.570991,0.860965,0.788037,0.463017,1.813201,1.403714,0.803287,1.103500
model_1_2_19,0.889581,0.728764,0.574211,0.861136,0.789144,0.463403,1.813754,1.393180,0.802298,1.097739
model_1_2_18,0.889479,0.728672,0.577455,0.861294,0.790251,0.463834,1.814372,1.382565,0.801383,1.091974
model_1_2_17,0.889365,0.728570,0.580723,0.861440,0.791359,0.464310,1.815056,1.371873,0.800543,1.086208
model_1_2_16,0.889241,0.728457,0.584014,0.861572,0.792466,0.464834,1.815809,1.361105,0.799781,1.080443
model_1_2_15,0.889104,0.728334,0.587327,0.861689,0.793573,0.465407,1.816633,1.350263,0.799101,1.074682


DataFrame salvo em ../results/metrics_1_2
+++++++++++ [1_3] | 1 ++++++++++++++++++
Elapsed time:  0.34797016300035466
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_3] | 2 ++++++++++++++++++
Elapsed time:  0.18577089000018532
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_3] | 3 ++++++++++++++++++
Elapsed time:  0.1578117160001966
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_3] | 4 ++++++++++++++++++
Elapsed time:  0.1562396730000728
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_3] | 5 ++++++++++++++++++
Elapsed time:  0.17189316000030885
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_3] | 6 ++++++++++++++++++
Elapsed time:  0.1639595950000512
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_3] | 7 ++++++++++++++++++
Elapsed time:  0.155657937

...

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_8_6_24,0.979034,0.790253,0.896260,0.984881,0.951177,0.087989,1.402580,0.487565,0.061582,0.274574
model_8_6_23,0.979166,0.790252,0.896961,0.984964,0.951500,0.087437,1.402588,0.484269,0.061244,0.272756
model_8_6_22,0.979310,0.790249,0.897736,0.985055,0.951857,0.086831,1.402604,0.480626,0.060874,0.270750
model_8_6_21,0.979468,0.790245,0.898592,0.985154,0.952250,0.086168,1.402633,0.476604,0.060470,0.268537
model_8_6_20,0.979641,0.790238,0.899536,0.985262,0.952684,0.085443,1.402678,0.472168,0.060030,0.266099
model_8_6_19,0.979829,0.790229,0.900576,0.985379,0.953161,0.084652,1.402743,0.467278,0.059552,0.263415
model_8_6_18,0.980034,0.790215,0.901722,0.985506,0.953686,0.083792,1.402835,0.461893,0.059035,0.260464
model_8_6_17,0.980256,0.790196,0.902983,0.985643,0.954262,0.082859,1.402961,0.455969,0.058477,0.257223
model_8_6_16,0.980497,0.790170,0.904367,0.985790,0.954894,0.081851,1.403131,0.449462,0.057878,0.253670
model_8_6_15,0.980755,0.790137,0.905886,0.985947,0.955585,0.080766,1.403358,0.442322,0.057241,0.249781


DataFrame salvo em ../results/metrics_8_6
+++++++++++ [8_7] | 1 ++++++++++++++++++
Elapsed time:  0.4052153229999931
Stopped at epoch:  15
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_7] | 2 ++++++++++++++++++
Elapsed time:  0.1723745709996365
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_7] | 3 ++++++++++++++++++
Elapsed time:  0.1781305690001318
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [8_7] | 4 ++++++++++++++++++
Elapsed time:  0.15965885600007823
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [8_7] | 5 ++++++++++++++++++
Elapsed time:  0.15770743900020534
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_7] | 6 ++++++++++++++++++
Elapsed time:  0.15791967900031523
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_7] | 7 ++++++++++++++++++
Elapsed time:  0.16179779

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_8_7_0,0.990062,0.777115,0.978067,0.900422,0.943951,0.041707,1.490431,0.049816,0.177425,0.113620
model_8_7_1,0.990259,0.776151,0.977051,0.901049,0.943658,0.040880,1.496881,0.052124,0.176307,0.114215
model_8_7_2,0.990401,0.775262,0.976090,0.901477,0.943307,0.040283,1.502827,0.054306,0.175545,0.114925
model_8_7_3,0.990501,0.774444,0.975187,0.901751,0.942922,0.039866,1.508291,0.056356,0.175057,0.115706
model_8_7_4,0.990567,0.773696,0.974344,0.901909,0.942519,0.039587,1.513298,0.058270,0.174776,0.116523
model_8_7_5,0.990608,0.773011,0.973561,0.901979,0.942111,0.039414,1.517875,0.060049,0.174651,0.117350
model_8_7_6,0.990630,0.772387,0.972837,0.901985,0.941708,0.039322,1.522051,0.061695,0.174640,0.118168
model_8_7_7,0.990638,0.771818,0.972168,0.901944,0.941315,0.039290,1.525855,0.063213,0.174714,0.118964
model_8_7_8,0.990635,0.771300,0.971553,0.901869,0.940938,0.039302,1.529315,0.064610,0.174846,0.119728
model_8_7_9,0.990625,0.770831,0.970989,0.901773,0.940580,0.039347,1.532457,0.065891,0.175018,0.120455


DataFrame salvo em ../results/metrics_8_7
+++++++++++ [8_8] | 1 ++++++++++++++++++
Elapsed time:  0.16095491299984133
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_8] | 2 ++++++++++++++++++
Elapsed time:  0.16428347099963503
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_8] | 3 ++++++++++++++++++
Elapsed time:  0.2933875349999653
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_8] | 4 ++++++++++++++++++
Elapsed time:  0.1925891410000986
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_8] | 5 ++++++++++++++++++
Elapsed time:  0.15656744099987918
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_8] | 6 ++++++++++++++++++
Elapsed time:  0.15720332899991263
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_8] | 7 ++++++++++++++++++
Elapsed time:  0.16366103

: 